In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm

Importamos nuestros datos y calculamos los rendimientos diarios de cada accion

In [3]:
raiz = r"C:\Users\cegom\Downloads"

In [4]:
data = pd.read_excel(f"{raiz}/PreciosPortf.xlsx",index_col=0)

In [19]:
rend = data.pct_change().dropna()
rend.head()

,ASURB MF Equity,GFNORTEO MF Equity,PE&OLES* MF Equity,WALMEX* MF Equity,IENOVA* MF Equity
DATE,,,,,
2016-01-04,-0.022961,-0.033688,-0.007448,-0.014946,0.008847
2016-01-05,-0.022074,-0.000763,0.003127,-0.004202,-0.020554
2016-01-06,-0.004549,-0.000981,-0.021366,-0.006564,-0.007555
2016-01-07,-0.005432,-0.022809,0.001506,-0.056395,0.019312
2016-01-08,-0.004118,-0.007706,-0.003180,0.026007,-0.017148


In [8]:
#Calculamos la esperanza anual del rendimiento de cada Accion
rend_ports = rend.mean()*252
rend_ports

ASURB MF Equity       0.123539
GFNORTEO MF Equity    0.078811
PE&OLES* MF Equity    0.111053
WALMEX* MF Equity     0.082426
IENOVA* MF Equity     0.078997
dtype: float64

In [9]:
#Matriz de Varianza-Covarianza
VarCov = rend.cov()

#### Definimos las funciones port_rend y port_vol para calcular la esperanza y desviacion estandar de un portafolio

In [10]:
def port_rend(weights, r):
    E_p = (r.mean() @ weights) * 252
    return E_p

def port_vol(weights, r):
    S_p = np.sqrt(weights.T @ (r.cov() * 252) @ weights)
    return S_p

#### Encontramos los pesos que minimicen el ratio de Sharpe

In [11]:
N, M = rend.shape
w0  = np.random.randn(M)
rf = 0.06

def sum_weights(weights):
    return weights.sum() - 1 # == 0

def func_sharpe(weights):
    return - (port_rend(weights, rend) - rf) / port_vol(weights, rend)

constraints = [
    {"type": "eq", "fun": sum_weights}
]

f_sharpe = minimize(func_sharpe, w0, constraints=constraints)

In [12]:
w_sharpe = f_sharpe.x

In [13]:
w_sharpe

array([ 0.73160497, -0.14132237,  0.17637294,  0.16460131,  0.06874315])

#### Se utilizarán los pesos optimizados de Sharpe para encontrar la Varianza y Desviacion Estandar Diaria y Anual del portafolio

In [16]:
varianza_diaria = np.dot(w_sharpe, np.dot(VarCov,w_sharpe.T))
desvstd_diaria = np.sqrt(varianza_diaria)

In [17]:
desvstd_anual = desvstd_diaria*np.sqrt(252)
varianza_anual = desvstd_anual**2

#### De la matriz de Var-Cov extraemos la varianza de cada accion individual para encontrar el nivel de diversificación

In [18]:
var_indiv = []
for i in range(VarCov.shape[0]):
    var_indiv.append(VarCov.values[i,i])
    

#### Funcion VaR: Obtendrá VaR, Diversificacion Anual y Diaria para un portafolio

Para utilizar funcion VaR necesitamos definir el monto a invertir, el nivel de confianza, desviacion estandar diaria y anual, pesos, matriz de Var_Cov y la varianza individual de cada accion

In [45]:
Inversion = 10000000
Confianza = 0.95

def VaR(Inversion, Confianza, desvstd_diaria, desvstd_anual, w_sharpe, VarCov, var_indiv):
    
    SN = norm.ppf(Confianza)
    VarAnn = Inversion * SN * desvstd_anual
    VarDaily = Inversion * SN * desvstd_diaria
    
    Diversificacion_Diaria = np.sum(w_sharpe * Inversion * np.sqrt(var_indiv) * SN) - VarDaily
    Diversificacion_Anual = np.sum(w_sharpe * Inversion * np.sqrt(var_indiv) * np.sqrt(252) * SN) - VarAnn
    
    return(f"VaR Anual: ${VarAnn:0.2f} , Diversificación Anual: ${Diversificacion_Anual:0.2f}; VaR Diario: {VarDaily: 0.2f}, Diversificación Diaria: {Diversificacion_Diaria: 0.2f}")
    


In [46]:
VaR(Inversion, Confianza, desvstd_diaria, desvstd_anual, w_sharpe, VarCov, var_indiv)

'VaR Anual: $3259184.51 , Diversificación Anual: $894730.91; VaR Diario:  205309.33, Diversificación Diaria:  56362.75'